In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import rand, col, count
from pyspark import StorageLevel
import time


# Crear un DataFrame de ejemplo
df = spark.range(0, 10_000_000).withColumn("valor", (rand() * 1000).cast("int"))
df_filtrado = df.filter(col("valor") > 500)

# Sin cache/persist - Primera acción
start = time.time()
df_filtrado.select("valor").groupBy("valor").agg(count("*")).count()
end = time.time()
print(f"Tiempo SIN cache/persist: {end - start:.2f} segundos")

# Con cache
df_cache = df.filter(col("valor") > 500).cache()

# Acción para activar la cache
df_cache.count()

# Repetir una acción sobre el mismo DataFrame
start = time.time()
df_cache.select("valor").groupBy("valor").agg(count("*")).count()
end = time.time()
print(f"Tiempo CON cache: {end - start:.2f} segundos")

# Liberar memoria
df_cache.unpersist()

# Con persist (solo memoria)
df_persist = df.filter(col("valor") > 500).persist(StorageLevel.MEMORY_ONLY)

# Acción para activar la persistencia
df_persist.count()

# Repetir otra acción
start = time.time()
df_persist.select("valor").groupBy("valor").agg(count("*")).count()
end = time.time()
print(f"Tiempo CON persistencia (MEMORY_ONLY): {end - start:.2f} segundos")

# Finalizar
df_persist.unpersist()


# 🧪 Ejercicio Práctico – Join y Persistencia en PySpark

## 🎯 Objetivo
Aplicar conceptos de joins entre múltiples tablas y explorar el impacto en el rendimiento al utilizar `cache()` y `persist()`.

---

## 📝 Instrucciones

### 1️⃣ Cargar las tablas del catálogo TPCH
Utilizá las siguientes tablas disponibles en Databricks:
- `samples.tpch.customer`
- `samples.tpch.nation`
- `samples.tpch.region`

### 2️⃣ Realizar joins
Uní las tres tablas siguiendo estas condiciones:
- `customer` → `nation` usando `c_nationkey == n_nationkey`
- `nation` → `region` usando `n_regionkey == r_regionkey`

Aplicá un **filtro** para quedarte solo con los clientes cuya región sea **'AMERICA'**.

```python
df_customer = spark.table("samples.tpch.customer")
df_nation = spark.table("samples.tpch.nation")
df_region = spark.table("samples.tpch.region")
```

---

### 3️⃣ Medir tiempo sin persistencia

---

### 4️⃣ Medir tiempo usando `cache()`

---

### 5️⃣ Medir tiempo usando `persist()`

---

## 📂 Entrega

Subí un notebook `.ipynb` o script `.py` con:
- El código completo
- Los tiempos registrados
- Una breve reflexión comparando `cache()` y `persist()`

---

## ✅ Criterios de evaluación

- ✔ Correcta aplicación de joins y filtros
- ✔ Uso adecuado de `cache()` y `persist()`
- ✔ Medición de tiempos clara
- ✔ Análisis final con observaciones propias

---

💡 *Tip:* Asegurate de que el clúster esté activo y no saturado para obtener mediciones consistentes.
